# Foundations of Computer Science Project

Project made by Daniele Monterisi [853257]

Data science - Università Bicocca

### Importing Modules

In the following chunk we import the modules we need in order to execute the requests.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import time
import random

## 1 - Normalize the loan_lenders table

In the normalized table, each row must have one loan_id and one lender.

First of all, we import the loan_lenders dataset.

### Importing loan_lenders Dataset

In [2]:
loan_lenders = pd.read_csv('loans_lenders.csv')

In [3]:
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


As we can see, for each loan_id there are more lenders. 


In order to get a table in wich each row has one load_id and one lender, we need to:
- split the attribute "lenders" to get a list of lenders
- separate every lender to obtain one loan_id and one lender by the explode method.

In [4]:
start = time.time()

#Split by " , " to avoid that " A" and "A" are two different strings.
loan_lenders_normalized = (loan_lenders.set_index(['loan_id'])
                   .apply(lambda x: x.str.split(', ').explode())
                   .reset_index())

end = time.time()

#Print the execution time of this code
print("Execution time: ", end - start)

Execution time:  23.249425888061523


In [5]:
loan_lenders_normalized

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


Now we have one lender for each loand id.

## 2- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time

The information about the disburse time and planned expiration time for each loan are stored in loans dataset.

### Importing loans dataset

In [6]:
loans = pd.read_csv('loans.csv')

In order to get the number of days between the disburse time and the planned expiration time we need to convert these attributes to datetime and to consider only the dates. 

We use the following function to convert the attributes.

In [7]:
#function to convert the attribute of the table to datetime

def convertTime(df, col):
    return pd.to_datetime(df[col]).dt.date

We create another attribute in loans dataset defined as the difference between planned_expiration_time and disburse_time.

In [8]:
start = time.time()


#Get difference between the disburse time and the planned expiration time

loans["duration"] =  convertTime(loans, "planned_expiration_time") - convertTime(loans, "disburse_time")

end = time.time()

#Print the execution time of this code
print("Execution time: ", end - start)

Execution time:  6.736309766769409


In [9]:
#view the head of the table

loans[["loan_id", "duration"]]

,loan_id,duration
0,657307,54 days
1,657259,96 days
2,658010,37 days
3,659347,35 days
4,656933,58 days
...,...,...
1419602,988180,40 days
1419603,988213,39 days
1419604,989109,51 days
1419605,989143,63 days


Now we have a new attribute "days" in the dataset.

## 3 - Find the lenders that have funded at least twice

We note that every loan has an attribute called status. In order to obtain the number of lenders who have lent at least twice we can exclude loans refunded or expired.

In [10]:
loans.status.value_counts()

funded         1350340
expired          59081
refunded          6578
fundRaising       3608
Name: status, dtype: int64

In order to obtain the set of lenders that have funded at least twice we need to marge loans and loan_lenders datasets and check the 'status' column. If the loan is funded we can select the lenders that have funded more than two time.

So we merge the set of funded loans ans the loan_lenders table.

In [11]:
loans_funded = pd.merge(loans["loan_id"].loc[loans["status"]=='funded'], loan_lenders_normalized, on = "loan_id")

In [12]:
loans_funded

,loan_id,lenders
0,657307,spencer5657
1,657307,matthew8640
2,657307,larry71496105
3,657259,ltr
4,657259,andrew5306
...,...,...
27145274,989240,jensdamsgaardvanar
27145275,989240,david47598776
27145276,989240,bernd9221
27145277,989240,valeria98599473


Now we can obtain the lenders who have funded at least twice.

In [13]:
loans_funded['lenders'].value_counts().loc[lambda x:x>=2]

gooddogg1           167512
trolltech4460       141268
gmct                123164
nms                  96713
themissionbeltco     74067
                     ...  
alma8811                 2
tyler65876925            2
samantha8384             2
lan8853                  2
dan5566                  2
Name: lenders, Length: 893981, dtype: int64

## 4 - For each country, compute how many loans have involved that country as borrowers

To compute how many loans have involved each country, we can count the occurrences of each coutry in loans dataset.

In [14]:
loans['country_name'].value_counts()

Philippines         285336
Kenya               143699
Peru                 86000
Cambodia             79701
El Salvador          64037
                     ...  
Papua New Guinea         1
Botswana                 1
Mauritania               1
Uruguay                  1
Canada                   1
Name: country_name, Length: 96, dtype: int64

## 5 - For each country, compute the overall amount of money borrowed

To compute the overall amount of monew borrowed for each country, we can group by country name and sum the loan amount.

In [15]:
country_amount = loans.groupby(['country_name'])['loan_amount'].sum().reset_index()

country_amount

,country_name,loan_amount
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0
...,...,...
91,Vietnam,24681100.0
92,Virgin Islands,10000.0
93,Yemen,3444000.0
94,Zambia,1978975.0


## 6-  Like the previous point, but expressed as a percentage of the overall amount lent

In [16]:
country_amount["percentage"] = country_amount["loan_amount"]/country_amount["loan_amount"].sum()*100

country_amount

,country_name,loan_amount,percentage
0,Afghanistan,1967950.0,0.166573
1,Albania,4307350.0,0.364586
2,Armenia,22950475.0,1.942589
3,Azerbaijan,14784625.0,1.251410
4,Belize,150175.0,0.012711
...,...,...,...
91,Vietnam,24681100.0,2.089074
92,Virgin Islands,10000.0,0.000846
93,Yemen,3444000.0,0.291509
94,Zambia,1978975.0,0.167506


## 7- Like the three previous point, but split for each year

-  For each country and year, compute how many loans have involved that country as borrowers

In [17]:
loans["disburse_year"]= pd.to_datetime(loans["disburse_time"]).dt.year

In [18]:
loans.groupby(['country_name', 'disburse_year']).size()

country_name  disburse_year
Afghanistan   2007.0            408
              2008.0            370
              2009.0            678
              2010.0            632
              2011.0            247
                               ... 
Zimbabwe      2013.0            426
              2014.0           2078
              2015.0            600
              2016.0            808
              2017.0           1079
Length: 748, dtype: int64

- For each country, compute the overall amount of money borrowed per year

In [19]:
loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum()

country_name  disburse_year
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            585125.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1311575.0
              2015.0            723625.0
              2016.0            788600.0
              2017.0           1237600.0
Name: loan_amount, Length: 748, dtype: float64

- Like the previous point, but expressed as a percentage of the overall amount lent

In [20]:
(loans.groupby(['country_name', 'disburse_year'])['loan_amount'].sum() / loans['loan_amount'].sum()) * 100

country_name  disburse_year
Afghanistan   2007.0           0.016503
              2008.0           0.030926
              2009.0           0.049527
              2010.0           0.047683
              2011.0           0.020748
                                 ...   
Zimbabwe      2013.0           0.057432
              2014.0           0.111015
              2015.0           0.061250
              2016.0           0.066749
              2017.0           0.104754
Name: loan_amount, Length: 748, dtype: float64

## 8 - For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

- First of all we merge loans table and loans_lender table. 
- Then we compute the contribution of each member per loan (assuming that each lender contribuited the same amount)
- Finally, we group by lenders and sum the amount 

In [21]:
lent_lenders = pd.merge(loan_lenders_normalized, loans[['loan_id', 'loan_amount', 'num_lenders_total']], on = 'loan_id')

In [22]:
lent_lenders["lender_contribution"] = lent_lenders['loan_amount']/lent_lenders['num_lenders_total']

In [23]:
lent_lenders.groupby('lenders')["lender_contribution"].sum().reset_index(name='lent')

,lenders,lent
0,000,1563.881085
1,00000,1249.947363
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1383794,zzmcfate,60150.832762
1383795,zzpaghetti9994,44.247788
1383796,zzrvmf8538,551.829575
1383797,zzzsai,236.888048


## 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Now we need the lenders dataset.

In [24]:
lenders = pd.read_csv('lenders.csv')

We need to handle the NaN value in country_code column. 

First of all we need this function to compute the distribution of a country:

In [25]:
def found_distribution(country_code):
    random.seed(1234)
    results = list()
    #country_code unique values without na
    values = pd.unique(country_code.dropna())
    #append these values to results list
    results.append(values)
    #prob computed as occurrences of country/number of country code
    probs = country_code.dropna().value_counts() / len(country_code.dropna())
    #append these probs to results list
    results.append(probs)
    #index of row where the country code is null
    index_rows_nan = country_code.index[country_code.isnull()]
    #append these index to results list
    results.append(index_rows_nan)
    #n is the number of rows with nan
    n = len(index_rows_nan)
    #append these value to results list
    results.append(n)
    return (results)

With this function we can found the distribution. 

In [26]:
results_function = found_distribution(lenders['country_code'])

To obtain a random country code with a distribution computer with found_distribution function we use a random.choice() that needs:

- array: a random sample is generated from its elements
- probabilities: the probabilities associated with each entry in the array.
- size: output shape

In [27]:
random_country_code = pd.Series(np.random.choice(results_function[0], p = results_function[1],
                                                size = results_function[3]), index = results_function[2])

Now we fill the na values with the random_country_code.

In [28]:
lenders['country_code'].fillna(random_country_code, inplace = True)

In [29]:
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,US,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,PR,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,ES,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,US,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,IN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,US,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,US,1342100213,NaN,NaN,NaN,NaN,0


Now we can compute the overall amount of money borrowed.

In [30]:
borrowed_by_country = loans.groupby('country_code')['loan_amount'].sum().reset_index(name = 'borrowed')

Now we can compute the overall amount of money lent. We need to merge the _lent_lenders_ dataset and the _lenders_ dataset. We need to find the attributes to do the merge.

In [31]:
lenders.columns

Index(['permanent_name', 'display_name', 'city', 'state', 'country_code',
       'member_since', 'occupation', 'loan_because', 'loan_purchase_num',
       'invited_by', 'num_invited'],
      dtype='object')

In lenders dataset, we have "permanent_name" that identifies the lenders.

In [32]:
lent_lenders.columns

Index(['loan_id', 'lenders', 'loan_amount', 'num_lenders_total',
       'lender_contribution'],
      dtype='object')

In lent_lenders dataset, we have "lenders" that identifies the lenders.

Now we can do the merge.

In [33]:
lent_lenders = pd.merge(lent_lenders, lenders[['permanent_name', 'country_code']], 
                             left_on = 'lenders', right_on = 'permanent_name')

Now we calculate the overall amount of money lent.

In [34]:
lent_country = lent_lenders.groupby('country_code')["lender_contribution"].sum().reset_index(name = 'lent')

Now we merge the borrowed_by_country and lent_country

In [35]:
lent_borrowed_by_country = pd.merge(lent_country, borrowed_by_country, on = 'country_code', 
                                    how = 'outer')

We fill the na values

In [36]:
lent_borrowed_by_country.fillna(0, inplace = True)

Now we compute the difference

In [37]:
lent_borrowed_by_country['difference'] = lent_borrowed_by_country['lent'] - lent_borrowed_by_country['borrowed']

In [38]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [39]:
lent_borrowed_by_country

,country_code,lent,borrowed,difference
0,AD,18774.26873,0.00000,18774.26873
1,AE,1748852.16533,0.00000,1748852.16533
2,AF,116477.68933,1967950.00000,-1851472.31067
3,AG,1031.54717,0.00000,1031.54717
4,AI,1704.79130,0.00000,1704.79130
...,...,...,...,...
229,YT,5004.80274,0.00000,5004.80274
230,ZA,499039.90958,1006525.00000,-507485.09042
231,ZM,35616.11306,1978975.00000,-1943358.88694
232,ZW,7343.55615,5851875.00000,-5844531.44385


## 10 - Which country has the highest ratio between the difference computed at the previous point and the population

At this point we need to read country_stat dataset to get information about the population.

In [40]:
country_stats = pd.read_csv('country_stats.csv')

In [41]:
country_stats.columns

Index(['country_name', 'country_code', 'country_code3', 'continent', 'region',
       'population', 'population_below_poverty_line', 'hdi', 'life_expectancy',
       'expected_years_of_schooling', 'mean_years_of_schooling', 'gni',
       'kiva_country_name'],
      dtype='object')

We merge the lent_borrowed_by_country dataset and the country_stats dataset on country_code. We need the "populaton_below_poverty_line" in the next point.

In [42]:
lent_borrowed_by_country = pd.merge(lent_borrowed_by_country, country_stats[['country_code', 'population', 
                                    'population_below_poverty_line']], 
                                     on = 'country_code')

In [43]:
lent_borrowed_by_country = lent_borrowed_by_country.set_index('country_code')

In [44]:
(lent_borrowed_by_country['difference'] / lent_borrowed_by_country['population']).idxmax()

'NO'

"NO" country has the highest ratio between the difference and the population.

## 11 - Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line

Since the population below the poverty line is expressed as a percentage, we get the percentage of people not in poverty in this way:

In [45]:
lent_borrowed_by_country['population_not_below_poverty_line'] = 100 - lent_borrowed_by_country['population_below_poverty_line']

In [46]:
lent_borrowed_by_country['abs_population_not_below_poverty_line'] = (lent_borrowed_by_country['population'] * lent_borrowed_by_country['population_not_below_poverty_line']) / 100

In [47]:
lent_borrowed_by_country.head()

,lent,borrowed,difference,population,population_below_poverty_line,population_not_below_poverty_line,abs_population_not_below_poverty_line
country_code,,,,,,,
AE,1748852.16533,0.00000,1748852.16533,9400145,19.50000,80.50000,7567116.72500
AF,116477.68933,1967950.00000,-1851472.31067,35530081,35.80000,64.20000,22810312.00200
AL,21953.94393,4307350.00000,-4285396.05607,2930187,14.30000,85.70000,2511170.25900
AM,69191.01234,22950475.00000,-22881283.98766,2930450,32.00000,68.00000,1992706.00000
AO,47005.63545,0.00000,47005.63545,29784193,40.50000,59.50000,17721594.83500


In [48]:
(lent_borrowed_by_country['difference'] / lent_borrowed_by_country['abs_population_not_below_poverty_line']).idxmax()

'US'

"US" country has the highest ratio between the difference and the population that is not below the poverty line.

## 12- For each year, compute the total amount of loans. 

Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.


In order to obtain the number of days between the begin date and the end date we use the following function.

- The parameter of this function are the start date and the end date of each loan.
- We create a list with the first date, every January 1st of years between start date and end date, end the end date.
- We create a key-values structure in wich the key is the year and the value the days between the current key and the next key

In [49]:

def day_calc(date1, date2):
    
    #control to avoid negative values
    (begin,end) = (date1,date2) if date1<date2 else (date2,date1) 
    
    #crate a list
    date_list = list() #primo valore data di inizio, secondo valore 1 gennaio anno successivo... fino alla data di fine
    
    #the first element of the list is the begin date
    date_list.append(begin)
    
    #for each year between begin date and end date append 1/01/year
    for y in range(begin.year+1, end.year+1): 
        date_list.append(pd.Timestamp(year=y, month=1, day=1, tz='utc')) 
    
    #append end date
    date_list.append(end) 
    
    #create a dictionary
    res = dict()
    
    #the years become the key and the value is the difference (in days) between i+1 key and i key
    for i in range(len(date_list)-1):
        res[date_list[i].year] = (date_list[i+1] - date_list[i]).days 
    
    #create the serie in wich the index is the key
    return pd.Series(res, name='DateValue')

We need to use the attribute _planned_expiration_time_ and the attributo _disburse_time_ as date, so we convert them to datetime.

In [50]:
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration,disburse_year
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.00000,125.00000,funded,General Store,Retail,...,3,2,1,NaN,female,true,irregular,field_partner,54 days,2013.00000
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.00000,400.00000,funded,Used Clothing,Clothing,...,11,2,1,NaN,female,true,monthly,field_partner,96 days,2013.00000
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.00000,400.00000,funded,General Store,Retail,...,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days,2014.00000
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.00000,625.00000,funded,Farming,Agriculture,...,21,2,1,user_favorite,female,true,monthly,field_partner,35 days,2014.00000
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.00000,425.00000,funded,Farming,Agriculture,...,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,58 days,2013.00000


In [51]:
#drop na value from loan
loans_drop = loans.dropna()
loans_drop['duration'] = loans_drop['duration'].dt.days
#for each loan in loans_drop 
days = loans_drop.apply(lambda x: day_calc(x['planned_expiration_time'],x['disburse_time']), axis = 1) 
days.head()

D:\Software\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,2011,2012,2013,2014,2015,2016,2017,2018
3,nan,nan,nan,34.00000,nan,nan,nan,nan
9,nan,nan,nan,68.00000,nan,nan,nan,nan
11,nan,nan,nan,36.00000,nan,nan,nan,nan
13,nan,nan,5.00000,84.00000,nan,nan,nan,nan
18,nan,nan,nan,nan,46.00000,nan,nan,nan


In [52]:
#fill na values with 0
days.fillna(0, inplace= True)
#concat loan_id, òpan_amount and duration
days = pd.concat([days, loans_drop['loan_id']
                 , loans_drop['loan_amount']
                 , loans_drop['duration'] ]
                , axis = 1, sort = True)

days.head()

,2011,2012,2013,2014,2015,2016,2017,2018,loan_id,loan_amount,duration
3,0.00000,0.00000,0.00000,34.00000,0.00000,0.00000,0.00000,0.00000,659347,625.00000,35
9,0.00000,0.00000,0.00000,68.00000,0.00000,0.00000,0.00000,0.00000,662117,1825.00000,68
11,0.00000,0.00000,0.00000,36.00000,0.00000,0.00000,0.00000,0.00000,659929,950.00000,36
13,0.00000,0.00000,5.00000,84.00000,0.00000,0.00000,0.00000,0.00000,661165,300.00000,90
18,0.00000,0.00000,0.00000,0.00000,46.00000,0.00000,0.00000,0.00000,956960,375.00000,46


Now we can divide [2011,...,2018] by duration and multiply by loan_amount.
 - Example: in the previous table we have 5 days in 2013 and 85 days (considering the rounding) in 2014 (13 row). So, we can compute 5/90 x 300= 16,6 -> this is the amount of money for 2013. Now we can compute 85/90 x 300 = 283,4. The sum is 300.

In [53]:
amounts = days[[2011,2012,2013,2014,2015,2016,2017,2018
                        ]].div(days.duration, axis=0).mul(days.loan_amount, axis = 0)

amounts = pd.concat([days['loan_id'], amounts],  axis = 1 )  
amounts = amounts.melt(id_vars=["loan_id"], 
        var_name="Year", 
        value_name="amount_per_year")

amounts.groupby(['loan_id', 'Year']).sum()

amount_per_year
loan_id Year                 
372752  2011        180.00000
        2012        560.00000
        2013          0.00000
        2014          0.00000
        2015          0.00000
...                       ...
1443953 2014          0.00000
        2015          0.00000
        2016          0.00000
        2017        175.00000
        2018        700.00000

[3830696 rows x 1 columns]